# Model Selection
Nested cross-validation with stratified 10-fold cross-validation and grid search is used for model selection.

## Library Imports

In [ ]:
import os

import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

RAND_STATE = 0

## Importing the Train Set

In [ ]:
datasets_folder = f"{os.path.abspath(os.path.join(os.getcwd(), os.pardir))}/datasets"

In [ ]:
X_train = pd.read_csv(os.path.join(datasets_folder, "obesity_X_train.csv"), index_col=0)
X_train

In [ ]:
y_train = pd.read_csv(os.path.join(datasets_folder, "obesity_y_train.csv"), index_col=0)["NObeyesdad"]
y_train

## Models

Logistic regression, random forests, decision trees and support vector machines are selected for comparison.

In [ ]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=RAND_STATE),
    'Random Forest': RandomForestClassifier(random_state=RAND_STATE),
    'Decision Tree': DecisionTreeClassifier(random_state=RAND_STATE),
    'SVM': SVC(probability=True, random_state=RAND_STATE)
}

## Hyperparameter Grids

We will be using a grid search, so we define the hyperparameter grids for tuning each model.

In [ ]:
param_grids = {
    'Logistic Regression': {'C': [0.001, 0.01, 0.1, 1, 10, 100]},
    'Random Forest': {'n_estimators': [50, 100, 200, 300, 400], 'max_depth': [None, 5, 10, 20, 30]},
    'Decision Tree': {'max_depth': [None, 5, 10, 20, 30], 'min_samples_split': [2, 5, 10, 20]},
    'SVM': {'C': [0.01, 0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly']}
}

## Performing Nested Cross-Validation

We perform nested cross-validation using 10 folds (stratified) and a grid search for hyperparameter tuning.

In [ ]:
# Performs nested cross-validation for a model.
def cross_validate(model_name, model):
    # Use stratified k-fold with k = 10.
    inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=RAND_STATE)
    
    # Perform hyperparameter tuning using a grid search.
    search_cv = GridSearchCV(
        model,
        param_grids[model_name],
        cv=inner_cv,
        scoring='accuracy',
        n_jobs=-3 # Use 2 less than the number of CPUs
    )
    search_cv.fit(X_train, y_train.astype("int"))
    
    return model_name, search_cv

for model_name, search_cv in (cross_validate(model_name, model) for model_name, model in models.items()):
    print(f"Model: {model_name}")
    print(f"Best Score: {search_cv.best_score_:.3f}")
    print(f"Best Parameters: {search_cv.best_params_}")
    print()